In [1]:
from pyspark import SparkContext
import os

stop_words = ['they', 'she', 'he', 'it', 'the', 'as', 'is', 'and']
sc = SparkContext("local", "app")

21/12/02 02:18:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# get all files in the data folder
file_arr = []
def construct_rdd(root_dir, file_arr):
    for fname in os.listdir(root_dir):
        f = os.path.join(root_dir, fname)
        if os.path.isfile(f):
            file_arr.append(f)
        else:
            construct_rdd(f, file_arr)
    
    return file_arr

In [3]:
home_dir = '/notebooks/data'
construct_rdd(home_dir, file_arr)

['/notebooks/data/Tolstoy/anna_karenhina.txt',
 '/notebooks/data/Tolstoy/war_and_peace.txt',
 '/notebooks/data/test.txt',
 '/notebooks/data/Hugo/Miserables.txt',
 '/notebooks/data/Hugo/NotreDame_De_Paris.txt',
 '/notebooks/data/shakespeare/histories/kingrichardiii',
 '/notebooks/data/shakespeare/histories/kingrichardii',
 '/notebooks/data/shakespeare/histories/kinghenryv',
 '/notebooks/data/shakespeare/histories/kinghenryviii',
 '/notebooks/data/shakespeare/histories/kingjohn',
 '/notebooks/data/shakespeare/histories/2kinghenryiv',
 '/notebooks/data/shakespeare/histories/3kinghenryvi',
 '/notebooks/data/shakespeare/histories/1kinghenryiv',
 '/notebooks/data/shakespeare/histories/1kinghenryvi',
 '/notebooks/data/shakespeare/histories/2kinghenryvi',
 '/notebooks/data/shakespeare/poetry/sonnets',
 '/notebooks/data/shakespeare/poetry/rapeoflucrece',
 '/notebooks/data/shakespeare/poetry/various',
 '/notebooks/data/shakespeare/poetry/venusandadonis',
 '/notebooks/data/shakespeare/poetry/love

In [11]:
# create RDD in format ((word, file), count)
rdd = None
for f_path in file_arr:
    file = sc.wholeTextFiles(f_path)

    f_rdd = file.flatMap(lambda x: [(x[0], word) for word in x[1].lower().split()]) \
            .filter(lambda x: x[1] not in stop_words) \
            .map(lambda x: ((x[1],x[0]), 1)).reduceByKey(lambda a,b: a+b)
    
    if rdd:
        rdd = rdd.union(f_rdd)
    else:
        rdd = f_rdd

In [12]:
# create output RDD in format (word, [(file, count)])
output_rdd = rdd.map(lambda x: (x[0][0], (x[0][1], x[1]))).groupByKey().mapValues(list)
output_rdd.take(10)

[('project',
  [('file:/notebooks/data/Tolstoy/anna_karenhina.txt', 84),
   ('file:/notebooks/data/Tolstoy/war_and_peace.txt', 91),
   ('file:/notebooks/data/Hugo/Miserables.txt', 86),
   ('file:/notebooks/data/Hugo/NotreDame_De_Paris.txt', 85),
   ('file:/notebooks/data/shakespeare/histories/2kinghenryiv', 1),
   ('file:/notebooks/data/shakespeare/tragedies/antonyandcleopatra', 1),
   ('file:/notebooks/data/shakespeare/tragedies/hamlet', 1),
   ('file:/notebooks/data/shakespeare/comedies/muchadoaboutnothing', 1),
   ('file:/notebooks/data/shakespeare/comedies/tempest', 3),
   ('file:/notebooks/data/shakespeare/comedies/allswellthatendswell', 1),
   ('file:/notebooks/data/shakespeare/comedies/winterstale', 1),
   ('file:/notebooks/data/shakespeare/glossary', 1)]),
 ('gutenberg',
  [('file:/notebooks/data/Tolstoy/anna_karenhina.txt', 24),
   ('file:/notebooks/data/Tolstoy/war_and_peace.txt', 24),
   ('file:/notebooks/data/Hugo/Miserables.txt', 24),
   ('file:/notebooks/data/Hugo/NotreDa